In [ ]:
import os
arch = os.getenv("ARGS", "real")

In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import firedrake
else:
    try:
        import firedrake
    except ImportError:
        if arch != "complex":
            !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
        else:
            !wget "https://fem-on-colab.github.io/releases/firedrake-install-complex.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
        import firedrake

In [ ]:
import numpy as np
from petsc4py import PETSc
if arch != "complex":
    assert not np.issubdtype(PETSc.ScalarType, np.complexfloating)
else:
    assert np.issubdtype(PETSc.ScalarType, np.complexfloating)

In [ ]:
mesh = firedrake.UnitIntervalMesh(3)

In [ ]:
V = firedrake.FunctionSpace(mesh, "CG", 1)
assert V.dim() == 4

In [ ]:
u = firedrake.TrialFunction(V)
v = firedrake.TestFunction(V)

In [ ]:
inner = firedrake.inner
dx = firedrake.dx

In [ ]:
f = firedrake.Function(V)
f.vector()[:] = np.arange(1, V.dim() + 1)

In [ ]:
a = inner(u, v) * dx
F = inner(f, v) * dx

In [ ]:
A = firedrake.assemble(a)
b = firedrake.assemble(F)

In [ ]:
solution = firedrake.Function(V)

In [ ]:
firedrake.solve(A, solution.vector(), b)

In [ ]:
with solution.dat.vec as solution_vec:
    assert np.allclose(solution_vec.getArray(), np.arange(1, V.dim() + 1))

In [ ]:
ksp = PETSc.KSP().create()
ksp.setOperators(A.petscmat)
with b.dat.vec_ro as b_vec, solution.dat.vec as solution_vec:
    ksp.solve(b_vec, solution_vec)
with solution.dat.vec_ro as solution_vec:
    assert np.allclose(solution_vec.getArray(), np.arange(1, V.dim() + 1))
ksp.destroy()

In [ ]:
for package in ("mumps", "superlu", "superlu_dist"):
    ksp = PETSc.KSP().create()
    ksp.setOperators(A.petscmat)
    ksp.setType("preonly")
    pc = ksp.getPC()
    pc.setType("lu")
    pc.setFactorSolverType(package)
    with b.dat.vec_ro as b_vec, solution.dat.vec as solution_vec:
        ksp.solve(b_vec, solution_vec)
    with solution.dat.vec_ro as solution_vec:
        assert np.allclose(solution_vec.getArray(), np.arange(1, V.dim() + 1))
    ksp.destroy()

In [ ]:
grad = firedrake.grad

In [ ]:
k = inner(grad(u), grad(v)) * dx

In [ ]:
K = firedrake.assemble(k)

In [ ]:
expected = (0, 10.8, 54, 108)

In [ ]:
from slepc4py import SLEPc

In [ ]:
eps = SLEPc.EPS().create()
eps.setOperators(K.petscmat, A.petscmat)
eps.setProblemType(SLEPc.EPS.ProblemType.GHEP)
eps.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_REAL)
eps.solve()
assert eps.getConverged() == len(expected)
for (i, eig_i_ex) in enumerate(expected):
    eig_i = eps.getEigenvalue(i)
    assert np.isclose(eig_i.real, eig_i_ex)
    assert np.isclose(eig_i.imag, 0)
eps.destroy()

In [ ]:
for package in ("mumps", "superlu", "superlu_dist"):
    eps = SLEPc.EPS().create()
    eps.setOperators(K.petscmat, A.petscmat)
    eps.setProblemType(SLEPc.EPS.ProblemType.GHEP)
    eps.setWhichEigenpairs(SLEPc.EPS.Which.TARGET_REAL)
    eps.setTarget(1)
    st = eps.getST()
    st.setType(SLEPc.ST.Type.SINVERT)
    st.setShift(1)
    ksp = st.getKSP()
    ksp.setType("preonly")
    pc = ksp.getPC()
    pc.setType("lu")
    pc.setFactorSolverType(package)
    eps.solve()
    assert eps.getConverged() == len(expected)
    for (i, eig_i_ex) in enumerate(expected):
        eig_i = eps.getEigenvalue(i)
        assert np.isclose(eig_i.real, eig_i_ex)
        assert np.isclose(eig_i.imag, 0)
    eps.destroy()